In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import time
import os

In [35]:
# Import API key and Configure Maps
from config import vacation_api_key

gmaps.configure(api_key=vacation_api_key)

In [36]:
#Use Final WeatherPy dataframe as start for Heatmap and Vacation Query
vacation_df = pd.read_csv("Output/weather_output.csv")
#Check dataframe column types
vacation_df.dtypes

Unnamed: 0      int64
City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Long          float64
Max Temp      float64
Wind Speed    float64
dtype: object

In [37]:
#begin to create location coordinates and humidity for heatmap
locations = vacation_df[["Lat", "Long"]]
humidity = vacation_df["Humidity"].astype(float)                    

In [38]:
#Plotting Heatmap
fig = gmaps.figure(center=(50.0, -10.0), zoom_level=2)
#Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=1)
#Adding layer
fig.add_layer(heat_layer)
#Post Heatmap
fig


Figure(layout=FigureLayout(height='420px'))

SCREENSHOT of Heatmap above saved in /OUTPUT

In [39]:
#Begin to constrain vacation dataframe based on preferences
#I would like to go on a bike riding trip, I do not want it Windy or Hot.  Clouds are fine.
ideal_city_df = vacation_df.loc[(vacation_df["Wind Speed"] <= 7) & (vacation_df["Cloudiness"] < 10) & (vacation_df["Max Temp"] <= 75)].dropna()
ideal_city_df.head()                                                                                                        

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
20,20,jamestown,1,US,1599530033,93,42.10,-79.24,63.00,4.70
34,34,bredasdorp,0,ZA,1599530036,93,-34.53,20.04,50.00,2.24
36,36,mar del plata,0,ES,1599530036,87,43.40,-4.01,60.01,1.12
38,38,rodionovo-nesvetayskaya,0,RU,1599530037,50,47.61,39.71,55.40,2.24
40,40,pevek,0,RU,1599530037,78,69.70,170.31,42.06,3.13


In [40]:
#Narrow down the dataframe to include city, country, and coordinates.  Also adding Column for Hotels.
hotels_df = ideal_city_df.loc[:,["City", "Country", "Lat", "Long"]]
hotels_df["Hotel Name"] = ""
hotels_df.head()

,City,Country,Lat,Long,Hotel Name
20,jamestown,US,42.10,-79.24,
34,bredasdorp,ZA,-34.53,20.04,
36,mar del plata,ES,43.40,-4.01,
38,rodionovo-nesvetayskaya,RU,47.61,39.71,
40,pevek,RU,69.70,170.31,


In [41]:
#Begin to build the parameters for Google Search of Hotels
#setting up parameters
params = {
    "type" : "hotel",
    "keyword" : "hotel",
    "radius" : 5000,
    "key" : vacation_api_key
}
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [42]:
#Set up For loop to get Hotels that match city and Coordinates
for index, row in hotels_df.iterrows():
    lat = row["Lat"]
    lng = row["Long"]
    city_name = row["City"]
    #Adding the keyword to the original params list
    params["location"] = f"{lat},{lng}"
    #Setup url and Make the request
    print(f"Getting results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    #Get results
    results = response['results']
    #Print out the output and put the results in the hotels dataframe
    try:
        print(f"The nearest hotel in {city_name} is {results[0]['name']}.")
        hotels_df.loc[index, "Hotel Name"] = results[0]['name']
    #If there is no hotels in the city    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("------------------")
#Print when search is completed
print("-----End of Search-----")    

Getting results for Index 20: jamestown.
The nearest hotel in jamestown is Chautauqua Harbor Hotel.
------------------
Getting results for Index 34: bredasdorp.
The nearest hotel in bredasdorp is De Hoop Collection.
------------------
Getting results for Index 36: mar del plata.
The nearest hotel in mar del plata is Hotel Costa Esmeralda Suites.
------------------
Getting results for Index 38: rodionovo-nesvetayskaya.
Missing field/result... skipping.
------------------
Getting results for Index 40: pevek.
The nearest hotel in pevek is Munitsipal'naya Gostinitsa.
------------------
Getting results for Index 43: lithia springs.
Missing field/result... skipping.
------------------
Getting results for Index 46: mainz.
The nearest hotel in mainz is Domäne Mechtildshausen.
------------------
Getting results for Index 64: abu samrah.
The nearest hotel in abu samrah is Gorbandh Hotel.
------------------
Getting results for Index 70: talnakh.
The nearest hotel in talnakh is Talnakh Hotel.
----

The nearest hotel in mwene-ditu is Hôtel Kakangayi.
------------------
Getting results for Index 494: charlottesville.
The nearest hotel in charlottesville is Hyatt Place Charlottesville.
------------------
Getting results for Index 507: pafos.
The nearest hotel in pafos is Constantinou Bros Asimina Suites Hotel.
------------------
Getting results for Index 510: lumeje.
Missing field/result... skipping.
------------------
Getting results for Index 511: sharlyk.
The nearest hotel in sharlyk is Gostinitsa "Mayak".
------------------
Getting results for Index 513: khandyga.
The nearest hotel in khandyga is Prezidentskaya Gostinitsa.
------------------
Getting results for Index 522: meridian.
The nearest hotel in meridian is Tru by Hilton Meridian Boise West.
------------------
Getting results for Index 525: mahajanga.
The nearest hotel in mahajanga is Lodge des Terres Blanches.
------------------
Getting results for Index 535: atbasar.
The nearest hotel in atbasar is The Ritz-Carlton, Ast

In [43]:
hotels_df

,City,Country,Lat,Long,Hotel Name
20,jamestown,US,42.10,-79.24,Chautauqua Harbor Hotel
34,bredasdorp,ZA,-34.53,20.04,De Hoop Collection
36,mar del plata,ES,43.40,-4.01,Hotel Costa Esmeralda Suites
38,rodionovo-nesvetayskaya,RU,47.61,39.71,
40,pevek,RU,69.70,170.31,Munitsipal'naya Gostinitsa
...,...,...,...,...,...
511,sharlyk,RU,52.92,54.75,"Gostinitsa ""Mayak"""
513,khandyga,RU,62.67,135.60,Prezidentskaya Gostinitsa
522,meridian,US,43.61,-116.39,Tru by Hilton Meridian Boise West
525,mahajanga,MG,-15.72,46.32,Lodge des Terres Blanches


In [53]:
#Dropping all cities without a Hotel from API
hotels_df.drop(hotels_df.loc[hotels_df['Hotel Name']==""].index, inplace=True)

In [54]:
hotels_df #there are 67 cities with listed Hotels that meet the weather criteria

,City,Country,Lat,Long,Hotel Name
20,jamestown,US,42.10,-79.24,Chautauqua Harbor Hotel
34,bredasdorp,ZA,-34.53,20.04,De Hoop Collection
36,mar del plata,ES,43.40,-4.01,Hotel Costa Esmeralda Suites
40,pevek,RU,69.70,170.31,Munitsipal'naya Gostinitsa
46,mainz,DE,50.00,8.27,Domäne Mechtildshausen
...,...,...,...,...,...
511,sharlyk,RU,52.92,54.75,"Gostinitsa ""Mayak"""
513,khandyga,RU,62.67,135.60,Prezidentskaya Gostinitsa
522,meridian,US,43.61,-116.39,Tru by Hilton Meridian Boise West
525,mahajanga,MG,-15.72,46.32,Lodge des Terres Blanches


In [55]:
#Putting listed hotel locations over original heatmap
hotel_location_tuples=[]
for index, row in hotels_df.iterrows():
    hotel_location_tuples.append( tuple([row["Lat"],row["Long"]] ))
#Set the Tuple list to be the layer
markers=gmaps.marker_layer(hotel_location_tuples)
fig.add_layer(markers)
#Print revised Heatmap with Hotel markers
fig

Figure(layout=FigureLayout(height='420px'))

SCREENSHOT of Heatmap and Hotels above saved in /OUTPUT